In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: BasicAUD – A simple Audification of 1D data

This notebook introduces and demonstrates usage of the BasicAUD sonecule.
- The sonecule enables a simple Audification of a data series.
- It can be initialized with 
  - a pya Asig (i.e. audio signal using pya)
    - `BasicAUD(asig, sr=None, channels=None)`
  - a pandas DataFrame or Series
    - `BasicAUD.from_df(df, sr=44100, time_column=None, columns=None)`
  - a numpy ndarray
    - `BasicAUD.from_np(data, sr=44100, time_column=None, columns=None)`
- Preprocessing such as time stretching, slicing, filtering is offered by specialized functions, either in pya (iirfilter, stretch) or libraries such as scipy.signal - correspondingly processed signals can be passed into Audification modules for audition and interaction.
- The current BasicAUD uses SuperCollider3, controlled via sc3nb, as Backend and therein uses a PlayBuf UGen for audification, which allows one-shot, looped, or repeatedly triggered playback of a data buffer.
- The synth is mutable, i.e. its parameters can be controlled interactively (code or GUI)
  - should enable pause/resume
  - with rate control (note: not band-limited!)
  - with amp control
  - with pan control 
  - with onset (in schedule)
  - with (optional) BPF and BRF   bpcf, bprq, brcf brrq


Let's get started. First some imports and settings and startup of sonecules

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

In [ ]:
df = dataframes['eeg'].loc[:, [1,4,7,10]]
df.plot(subplots=True);
df.head()

## Usage Demo for the BasicAUD Sonecule

In [ ]:
from sonecules.bufferson import BasicAUD

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (aka Timeline)
- to start the Playback at a given rate
- to plot the Timeline.

In [ ]:
# create the sonecule from data (e.g. channel 7 of the EEG data set )
aud1 = BasicAUD.from_df(df, sr=256, columns=7)

# reset the timeline 
ctx.timeline.reset()

# schedule the event (which is just one: to start the synth)
aud1.schedule(at=0, rate=10, pan=0, loop=0, amp=0.5).start()

# plot the data (just for fun)
df.plot(subplots=True);

Once the BasicAUD instance is available, you can replay it as needed with different parameters:

In [ ]:
# btw, most arguments are optional
# rate default is 1 so we need to set it in order to hear these data
aud1.reschedule(rate=20).start()

In [ ]:
aud1.reschedule(at=0, rate=30, pan=1, loop=0, startpos=4000, amp=0.3).start()

In [ ]:
aud1.reschedule(at=0, rate=60, pan=-1, loop=0, startpos=0, amp=0.3).start()

Here is a very slow and low-frequency audification

In [ ]:
aud1.reschedule(at=0, rate=2, pan=0, loop=0, startpos=0, amp=1).start()

To stop the playing audification at any time use the following code (try it while the above line plays)

In [ ]:
with ctx.at(ctx.playback.time) as timepoint:  # used to specify the time point to stop the sonification
    print(f"Stopped aud1 at {timepoint}")
    aud1.stop()
# This can be also be seen in the 
ctx.timeline.plot()

To make this more iteractive we enable the realtime mode of the `Context`.

In [ ]:
aud1.reschedule(at=0, rate=2, pan=0, loop=0, startpos=0, amp=1)
ctx.enable_realtime(at=0);  # this will start the playback of the Timeline again

Now simply using `aud1.stop()` works as it assumes the `playback.time` as timepoint of it's execution

In [ ]:
aud1.stop()

Parameter modifications become more interesting as you loop the audification

In [ ]:
aud1.reschedule(at=0, rate=20, pan=1, loop=1, startpos=0, amp=0.8).start()

Now let's change the rate to 50 for faster temporal compression

In [ ]:
aud1.set(rate=50, amp=0.2, pan=-1)

In [ ]:
aud1.stop()

BasicAUD offers to set the start position `startpos` from which the playback starts.
- However, once the end is reached, it wraps around to the first sample, i.e. plays it does not move to startpos if the end is reached!

Furthermore BasicAUD offers a periodic trigger, at which the position is reset to `startpos`
- using loop=1, and a non-zero trfreq (the trigger rate in Hz), we can interactively skim through the file 

In [ ]:
# to start the sonification at a certain position, without looping
aud1.reschedule(at=0, rate=40, pan=1, loop=0, startpos=6000, amp=0.8).start()

In [ ]:
# let's start with loop and given trigger rate 'trfreq'
aud1.reschedule(at=0, rate=100, pan=0, loop=1, startpos=0, amp=0.8, trfreq=10).start() 

In [ ]:
pos = 0

In [ ]:
# run this cell a couple of times to step-by-step move forward
pos += 1000
aud1.set(startpos=pos)

In [ ]:
aud1.stop()

Now we can easily control the Audification with some sliders
* move the startpos slider to skim through the audification
* control rate and trigger rate independently

In [ ]:
from ipywidgets import interactive
aud1.reschedule(at=0, rate=100, pan=0, loop=1, startpos=0, amp=0.8, trfreq=10).start() 
def aud_gui(startpos=0, trfreq=10, rate=50):
    aud1.set(startpos=startpos, trfreq=trfreq, rate=rate)
interactive(aud_gui, startpos=(0, 12000, 100), trfreq=(1, 50, 1), rate=(1, 200, 1)) 

In [ ]:
# and stop when done
aud1.stop()

**Signal Conditioning**

* Basic Audification doesn't offer filtering or distortion, or multi-channel capabilities.
* These, however, could be made available in the future or more specialized Sonecules of the AUD family.
* A multi-channel audification is offered by MultivariateBasicAUD 
* However, some signal conditionings are better applied before audition anyway!
* Modifications such as applying a time scale modification (aka time stretching, i.e. rescaling the time without modifying the spectrum), is for instance well done in pya using Asig.stretch(factor) as shown here for a selected channel and time interval and stretch factor in a one-liner

      aud1 = BasicAUD(my_asig[{1.5:5.2},['channelname']].stretch(3.5))

* so while Sonecules probably don't do it all, combinations with pandas and pya functions enable swift, and flexible implementations of what is needed.

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# create or load your data, e.g.

# chaotic series from the logistic equation
a, x, xs = 3.5, 0.2, []
for i in range(30000): 
    x = a * x * (1-x); 
    a += 0.000016
    xs.append(x)
data = np.array(xs)

# or load data
# data = pd.read_csv("your_csv_file.csv", delimiter=",")
# data = pd.read_excel("your_excel_file.xlsc") # see pandas documenation
# put data series into an Asig
a1 = Asig(data, sr=10000)
plt.figure();a1.plot(lw=0.02)
a1

In [ ]:
# load your data / select your data
myasig = a1

# sonecule for your synth with defaults and bounds
aud1 = BasicAUD(a1)

# reset the timeline 
ctx.timeline.reset() 

# finally start the realtime playback at a given rate
aud1.schedule(at=0, rate=0.2, amp=0.2).start()

# if needed: plot the timeline using 
ctx.timeline.plot()

In [ ]:
ctx.close()